In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_draws_or_not.csv")

In [3]:
df.head(2)

,season,target,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
0,9,0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65,-0.841,-1.491
1,9,0,-15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,38.23,-3.330,-41.560


In [4]:
df['target'].value_counts()

0    4808
1    1652
Name: target, dtype: int64

In [5]:
4808/1652

2.9104116222760292

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [7]:
X_train.head(2)

,season,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
1600,13,11.0,-0.24,0.24,-0.13,-0.74,-6.375,-3.375,-0.067,-1.333,-1.2,-1.000,-0.667,-0.8,-0.715,7.0,-40.94,-1.270,39.670
4814,21,-7.0,-0.32,-0.64,0.24,0.28,-0.240,-0.120,-0.125,-2.000,-1.8,-1.286,-1.667,-2.2,-1.429,-4.0,11.57,-0.234,-11.804


In [8]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 0.21520896,  0.18209989, -0.00397309,  0.00397309, -0.00215209,
       -0.01225036, -0.10553517, -0.05587156, -0.00110915, -0.0220672 ,
       -0.01986544, -0.01655454, -0.01104188, -0.01324363, -0.01183649,
        0.11588175, -0.67774269, -0.02102426,  0.65671843])

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1,
                1: 2.91}

In [11]:
model.fit(X_train_norm, y_train.values, epochs=20, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 5168 samples, validate on 1292 samples
Epoch 1/20
5168/5168 [==============================] - 1s 253us/step - loss: 0.6933 - acc: 0.5584 - val_loss: 0.6833 - val_acc: 0.5952
Epoch 2/20
5168/5168 [==============================] - 1s 129us/step - loss: 0.6917 - acc: 0.5960 - val_loss: 0.6776 - val_acc: 0.6316
Epoch 3/20
5168/5168 [==============================] - 1s 116us/step - loss: 0.6911 - acc: 0.6395 - val_loss: 0.6884 - val_acc: 0.5372
Epoch 4/20
5168/5168 [==============================] - 1s 114us/step - loss: 0.6900 - acc: 0.6267 - val_loss: 0.6709 - val_acc: 0.6316
Epoch 5/20
5168/5168 [==============================] - 1s 142us/step - loss: 0.6891 - acc: 0.6080 - val_loss: 0.6732 - val_acc: 0.6014
Epoch 6/20
5168/5168 [==============================] - 1s 119us/step - loss: 0.6879 - acc: 0.5865 - val_loss: 0.6767 - val_acc: 0.5859
Epoch 7/20
5168/5168 [==============================] - 1s 115us/step - loss: 0.6870 - acc: 0.5869 - val_loss: 0.6759 - val_acc: 0.5805


In [12]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1]))

preds: {0: 3207, 1: 1961}
real: {0: 3846, 1: 1322}
[[2504  703]
 [1342  619]]


In [13]:
print(conf_matrix(test_pred,y_test.values))

[[620 186]
 [342 144]]


In [14]:
test_pred_probs = model.predict(X_test_norm)

In [15]:
test_pred_probs[0]

array([0.45320436, 0.5467956 ], dtype=float32)

In [16]:
2184/1264 #1

1.7278481012658229

In [17]:
2184/1269 #0

1.7210401891252955